In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import statsmodels
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix

In [0]:
from pylab import rcParams
rcParams['figure.figsize'] = 16, 8

In [0]:
tf.enable_v2_behavior()

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
x_train.shape

In [0]:
y_train.shape

In [0]:
plt.imshow(np.squeeze(x_train[-1]), cmap='Greys')

In [0]:
def CollapseDims(x):
  return x.reshape([x.shape[0], -1])

In [0]:
clf = LogisticRegression(random_state=0, C=0.01, tol=0.1).fit(
    X=CollapseDims(x_train), 
    y=y_train)

In [0]:
clf.score(CollapseDims(x_train), y_train)

In [0]:
clf.score(CollapseDims(x_test), y_test)

In [0]:
plot_confusion_matrix(clf, CollapseDims(x_test), y_test,
                                 cmap=plt.cm.Blues,
                                 normalize=None)

In [0]:
scale = np.abs(clf.coef_).max()
plt.imshow(clf.coef_[8].reshape(28, 28), interpolation='nearest',
                   cmap=plt.cm.RdBu, vmin=-scale, vmax=scale)
 

# Train a feed-forward network

In [0]:
img_rows, img_cols = 28, 28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=input_shape))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

In [0]:
score

In [0]:
model.count_params()

In [0]:
model.summary()

# Convolution

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten(input_shape=input_shape))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

In [0]:
model.count_params()

In [0]:
model.summary()

## Visualize first-layer filters

In [0]:
filter_num =  7#@param {type:"integer"}
image_index =  44#@param {type:"integer"}

In [0]:
plt.imshow(np.squeeze(model.layers[0].weights[0][:, :, :, filter_num]), cmap='coolwarm')

In [0]:
f, axarr = plt.subplots(1,2)
axarr[0].imshow(np.squeeze(x_train[-1]), cmap='Greys')
axarr[1].imshow(np.squeeze(tf.nn.conv2d(
    input=tf.expand_dims(x_train[-1], 0),
    filter=tf.expand_dims(model.layers[0].weights[0][:, :, :, filter_num], -1),
    padding='SAME')), cmap='Greys')